# Log In to Hugging Face

In [1]:
from kaggle_secrets import UserSecretsClient

ModuleNotFoundError: No module named 'kaggle_secrets'

In [3]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token")


ModuleNotFoundError: No module named 'kaggle_secrets'

In [2]:
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("Hhf_cOiGKVWvTMdAUcslMtBckHtZEXVtSwftMs")

NameError: name 'UserSecretsClient' is not defined

# Import Libraries

In [4]:
# from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import TextDataset, DataCollatorForLanguageModeling
import math

c:\Users\Acer\anaconda3\envs\dks\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Get Data

In [6]:
import os
file_path = os.path.join("data", "poem.txt")
poem_file = open(file_path, 'r')
poem = poem_file.read()

# poem_file = open('/kaggle/input/nepali-poem/poem.txt','r')
# poem = poem_file.read()

In [7]:
poem_corpus = poem.split("\n")
print(poem_corpus[:5])

['नछाडी जानोस् हे मेरा प्राण ! अकेली मलाई,', 'मनको वनमा ननिभ्ने गरी विरह जलाई !', 'ननिभ्ने गरी विरह जलाई,', 'लोचनका तारा ! हे मेर प्यारा ! यो जोति  बिलाए !', 'के भनूँ? भन्ने म केही थिइन  विष नै पिलाए !']


# Preprocess

In [8]:
def remove_noise(sentences):
    punctuations = ['\n','\ufeff','0','1','2','3','4','5','6','7','8','9','०','१','२','३','४','५','६','७','८','९','१०','\u200d']
    processed_sentences = []
    for sentence in sentences:
        for punct in punctuations:
            sentence = sentence.replace(punct,'')
        processed_sentences.append(sentence)

    return processed_sentences

In [9]:
processed_poem_corpus = remove_noise(poem_corpus)
print(processed_poem_corpus[:5])

['नछाडी जानोस् हे मेरा प्राण ! अकेली मलाई,', 'मनको वनमा ननिभ्ने गरी विरह जलाई !', 'ननिभ्ने गरी विरह जलाई,', 'लोचनका तारा ! हे मेर प्यारा ! यो जोति  बिलाए !', 'के भनूँ? भन्ने म केही थिइन  विष नै पिलाए !']


# Save

In [10]:
with open('processed_poem.txt','w') as f:
  for line in processed_poem_corpus:
    f.write(line + '\n')

# Set dataset Loader Pipeline

In [11]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator


# Train

In [12]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = AutoTokenizer.from_pretrained(model_name,padding=True, truncation=True, return_tensors="pt")
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = AutoModelForCausalLM.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
          report_to=["tensorboard"],
          push_to_hub=True,
          hub_token=hf_token
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
  trainer.train()
  trainer.save_model()

# Set output parameters

In [13]:
# you need to set parameters
train_file_path = "/kaggle/working/processed_poem.txt"
model_name = 'Sakonii/distilgpt2-nepali'
output_dir = 'nepali-poem-distilgpt2'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 250
save_steps = 1000

In [15]:
def train(train_file_path, model_name, output_dir, overwrite_output_dir, per_device_train_batch_size, num_train_epochs, save_steps):
    tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, return_tensors="pt")
    
    # Check if file exists
    if not os.path.exists(train_file_path):
        raise ValueError(f"Input file path {train_file_path} not found")

    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)
    
    tokenizer.save_pretrained(output_dir)
    # Your training code continues here...


In [16]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
)

ValueError: Input file path /kaggle/working/processed_poem.txt not found

In [17]:
def load_model(model_path):
    model = AutoModelForCausalLM.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = f"Bibek1129/{output_dir}"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [20]:
sequence = "हिमालको आकाश किनारापारि"
max_len = len(sequence)
generate_text(sequence, 20)

हिमालको आकाश किनारापारि मेरो छ घर एक कोस पर, तिमी त मर्
